In [ ]:
%%capture

! pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

# Question 1: Install Spark and PySpark

# Install Spark

# Run PySpark

# Create a local spark session

# Execute spark.version.

# What's the output?

In [ ]:
spark.version

In [ ]:
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [ ]:
df = spark.read \
    .option("header", "true") \
    .parquet("/content/yellow_tripdata_2024-10.parquet")

In [ ]:
df.printSchema()

# Question 2: Yellow October 2024 Read the October 2024 Yellow into a Spark Dataframe.

# Repartition the Dataframe to 4 partitions and save it to parquet.

# What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)?

In [ ]:
df = df.repartition(4)

In [ ]:
df \
    .write \
    .mode("overwrite") \
    .parquet("/content/pq/")

In [ ]:
! ls -ll pq

# Question 3: Count records

# How many taxi trips were there on the 15th of October?

# Consider only trips that started on the 15th of October.

In [ ]:
df \
    .withColumn("pickup_day", F.to_date(df.tpep_pickup_datetime)) \
    .filter("pickup_day = '2024-10-15'") \
    .count()

# Question 4: Longest trip

# What is the length of the longest trip in the dataset in hours?

In [ ]:
df \
    .withColumn("duration", F.to_timestamp(df.tpep_dropoff_datetime).cast("long") - F.to_timestamp(df.tpep_pickup_datetime).cast("long")) \
    .withColumn("pickup_day", F.to_date(df.tpep_pickup_datetime)) \
    .groupBy("pickup_day") \
    .max("duration") \
    .orderBy("max(duration)", ascending=False) \
    .show()

# Question 5: User Interface
# Spark’s User Interface which shows the application's dashboard runs on which local port?

4040

# Question 6: Least frequent pickup location zone

# Load the zone lookup data into a temp view in Spark
# Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [ ]:
! wget wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [ ]:
df_zones = spark.read \
    .option("header", "true") \
    .csv("/content/taxi_zone_lookup.csv")

In [ ]:
df_zones.printSchema()

In [ ]:
df.createOrReplaceTempView("yellow")

In [ ]:
df_zones.createOrReplaceTempView("zones")

In [ ]:
df_yellow_trips = spark.sql("""
select
    t.PULocationID as pickup_zone,
    z.Zone as zone_name,
    count(*) as cnt
from
    yellow as t
    inner join zones as z on t.PULocationID = z.LocationID
where
    tpep_pickup_datetime >= '2024-10-01 00:00:00'
group by
    PULocationID,
    z.Zone
order by
    count(*)
""")

In [ ]:
df_yellow_trips.show(truncate=False)